# Diff `measurements_old` and `measurements` layers

Connect to the database.

In [1]:
from estnltk.storage import PostgresStorage

storage = PostgresStorage(dbname='egcut_epi',
                          pgpass_file='~/.pgpass',
                          schema="grammarextractor",
                          role='egcut_epi_grammarextractor_create')

collection = storage.get_collection('psa_measurements_v22')

INFO:storage.py:99: connecting to host: '10.6.6.29', port: '5432', dbname: 'egcut_epi', user: 'dages'
INFO:storage.py:113: schema: 'grammarextractor', temporary: False, role: 'egcut_epi_grammarextractor_create'


In [2]:
texts = []
for idx, thing in enumerate(collection.select(layers = ['measurements'])):
    texts.append(thing)
    print(idx)
    break

KeyError: 6

Since `measurements` layer envelopes `measurement_tokens` layer but `measurements_old` layer does not, first flatten `measurements` layer into `measurements_flat` layer. As these layers have different attribute names, mapping of attribute names is also needed.

In [5]:
from estnltk.taggers import FlattenTagger

In [6]:
from estnltk.taggers import FlattenTagger

flatten_tagger = FlattenTagger(input_layer='measurements',
                               output_layer='measurements_flat',
                               #output_attributes=['name', 'key', 'value', 'unit', 'DATE'],
                               output_attributes=['name', 'value'],
                               attribute_mapping=(('REGEX_TYPE', 'name'), 
                                                  ('VALUE','value'), 
                                                  ('UNIT', 'unit')
                                                  )
                               )

collection.create_layer(tagger=flatten_tagger, overwrite=True, progressbar='notebook')

INFO:collection.py:856: collection: 'psa_measurements_v22'
INFO:collection.py:877: nothing to overwrite, preparing to create a new layer: 'measurements_flat'


KeyError: 6

Create layer of differences.

In [5]:
from estnltk.layer_operations import drop_annotations
from estnltk.storage.postgres import RowMapperRecord

In [6]:
def row_mapper_1(row):

    text_id, text = row[0], row[1]
    
    drop_annotations(layer=text.measurements_old,
                 attribute='name',
                 values={'TIMEX', 'PRAHT', 'PÕLETIK', 'EESNÄÄRME_BIOPSIA', 
                        'VABA_PSA_PROTSENT', 'PSA_ARVUDETA', 'PSA_KÕRGE', 'PSA_OK', 'VABA_PSA_ARVUDETA',
                         'NEUTROFIILID', 'LBBB', 'STEMI', 'NSTEMI', 'INFARKTI_KUUPÄEV', 
                         'KORONAARANGIOPLASTIKA', 'KORONAARANGIOGRAAFIA', 'BASOFIILID%', 
                        'EOSINOFIILID%', 'NEUTROFIILID%', 'MONOTSÜÜDID%', 'LÜMFOTSÜÜDID%', 'MPV%', 'PDW%',
                        'SUITS_JAH', 'SUITS_EI', 'TRUS', 'DRE', 'DRE_KAHELDAV', 'HINNAKOOD'},
                 function=None  # default: None
                 )
    for idx1, thing in enumerate(text.measurements_old):

        for idx2, value in enumerate(thing.value):
            if value:
                text.measurements_old[idx1][idx2].value = value.replace(',', '.').replace(' ', '').strip('.')
                
        for idx3, name in enumerate(thing.name):
            if name == 'VERESUHKUR':
                text.measurements_old[idx1][idx3].name = 'GLÜKOOS'
            elif name == 'BMI':
                text.measurements_old[idx1][idx3].name = 'KMI'
            elif name == 'FS-CRP':
                text.measurements_old[idx1][idx3].name = 'CRP'    
            else:
                text.measurements_old[idx1][idx3].name = name

    layer = text.measurements_old
    layer.name = 'measurements_old_new'

    return [RowMapperRecord(layer=layer, meta=None)]

In [ ]:
def fun():
    for idx, thing in enumerate(collection.select(layers = ['measurements_old'])):

        if len(thing[1].measurements_old) == 0:
            continue
        for span in thing[1].measurements_old:

            for annotation in span.annotations:
                if annotation.value != None:
                    if ',' in annotation.value:
                        #print(annotation.value)
                        return thing


In [8]:
collection.create_layer('measurements_old_new',
                                 data_iterator=collection.select(layers = ['measurements_old']),
                                 row_mapper=row_mapper_1, overwrite = True, progressbar='notebook')

INFO:collection.py:856: collection: 'psa_measurements_v22'
INFO:collection.py:877: nothing to overwrite, preparing to create a new layer: 'measurements_old_new'
INFO:collection.py:942: layer created: 'measurements_old_new'


In [9]:
flatten_tagger = FlattenTagger(input_layer='measurements_old_new',
                               output_layer='measurements_old_flat',
                               #output_attributes=['name', 'key', 'value', 'unit', 'DATE'],
                               output_attributes=['name', 'value', 'unit'],
                               #attribute_mapping=(('REGEX_TYPE', 'name'), 
                               #                   ('VALUE','value'), 
                               #                   )
                               )

collection.create_layer(tagger=flatten_tagger, overwrite=True, progressbar='notebook')

INFO:collection.py:856: collection: 'psa_measurements_v22'
INFO:collection.py:861: overwriting output layer: 'measurements_old_flat'
INFO:collection.py:1051: layer deleted: 'measurements_old_flat'



INFO:collection.py:942: layer created: 'measurements_old_flat'


In [13]:
from collections import OrderedDict
from estnltk.taggers import DiffTagger

diff_tagger = DiffTagger(layer_a='measurements_old_new',
                         layer_b='measurements_flat',
                         output_layer='measurements_diff_old_flat',
                         output_attributes=['name',  'value', 'unit']
                        )

In [12]:
meta = OrderedDict([('unchanged_annotations', 'int'),
                    ('missing_annotations', 'int'),
                    ('extra_annotations', 'int'),
                    ('unchanged_spans', 'int'),
                    ('modified_spans', 'int'),
                    ('missing_spans', 'int'),
                    ('extra_spans', 'int'),
                    ('conflicts', 'int'),
                    ('overlapped', 'int'),
                    ('prolonged', 'int'),
                    ('shortened', 'int')])

collection.create_layer(tagger=diff_tagger,
                                 overwrite=True,
                                 meta=meta,
                                 progressbar='notebook'
                                 )

INFO:collection.py:856: collection: 'psa_measurements_v22'
INFO:collection.py:877: nothing to overwrite, preparing to create a new layer: 'measurements_diff_old_flat'


PgCollectionException: there is no 'measurements_flat' layer in the collection 'psa_measurements_v22'

## Statistics
Tabel of diff layer meta data.

In [2]:
from estnltk.taggers.standard_taggers.diff_tagger import DiffSampler


sampler = DiffSampler(collection=collection, layer='measurements_diff_old_flat')

In [3]:
sampler.layer_meta.loc[:10]

,text_id,unchanged_annotations,missing_annotations,extra_annotations,unchanged_spans,modified_spans,missing_spans,extra_spans,conflicts,overlapped,prolonged,shortened,id
0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,0,0,2
2,3,0,0,0,0,0,0,0,0,0,0,0,3
3,4,2,1,1,2,0,1,1,1,0,0,1,4
4,5,0,0,0,0,0,0,0,0,0,0,0,5
5,6,0,1,1,0,0,1,1,1,0,0,1,6
6,7,0,0,0,0,0,0,0,0,0,0,0,7
7,8,0,0,0,0,0,0,0,0,0,0,0,8
8,9,0,0,0,0,0,0,0,0,0,0,0,9
9,10,1,0,0,1,0,0,0,0,0,0,0,10


In [4]:
sampler.layer_meta.sum()

text_id                  2239578201
unchanged_annotations          3482
missing_annotations           31803
extra_annotations             30425
unchanged_spans                3482
modified_spans                  153
missing_spans                 31650
extra_spans                   30272
conflicts                     26461
overlapped                       43
prolonged                        74
shortened                     26344
id                       2239578201
dtype: int64

## Random samples
The following iterations give different kind of random samlples without replacement. The domain names of samples (`'modified_spans'`, `missing_spans'` etc) correspond to the summary table columns.

In [5]:
for text_id, span_index, span in sampler.sample_spans(k=100, domain='modified_spans'):
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    span.display()

text_id: 971, span index: 0


text,start,end,span_status,input_layer_name,name,value
CRP 15,36,42,modified,measurements_old_flat,FS-CRP,15
,36,42,modified,measurements_flat2,CRP,15


text_id: 1047, span index: 0


text,start,end,span_status,input_layer_name,name,value
CRP-0,190,195,modified,measurements_old_flat,FS-CRP,0
,190,195,modified,measurements_flat2,CRP,0


text_id: 2952, span index: 0


...

text,start,end,span_status,input_layer_name,name,value
"CRP 0,01",13,22,modified,measurements_old_flat,FS-CRP,0.01
,13,22,modified,measurements_flat2,CRP,0.01


text_id: 65457, span index: 0


text,start,end,span_status,input_layer_name,name,value
"CRP 5,23",60,68,modified,measurements_old_flat,FS-CRP,5.23
,60,68,modified,measurements_flat2,CRP,5.23


text_id: 66107, span index: 0


text,start,end,span_status,input_layer_name,name,value
"CRP 0,12",78,86,modified,measurements_old_flat,FS-CRP,0.12
,78,86,modified,measurements_flat2,CRP,0.12


In [5]:
texts = []
for idx, thing in enumerate(collection.select(layers = ['measurements_old_new', 'measurements_flat2'])):
    texts.append(thing)

In [78]:
examples = []

In [79]:
examples.append([texts[48][1].text, 0, 20, texts[48][1].measurements_flat2.value[0][0]])

In [80]:
examples.append([texts[865][1].text, 0, 6, texts[865][1].measurements_old_new.value[0][0]])

In [81]:
examples.append([texts[3715][1].text, 0, 11, texts[3715][1].measurements_old_new.value[0][0]])

In [82]:
examples.append([texts[5697][1].text, 0, 23, texts[5697][1].measurements_old_new.value[0][0]])

In [83]:
examples.append([texts[8913][1].text, 581, 640, texts[8913][1].measurements_old_new.value[0][0]])

In [90]:
examples.append([texts[10211][1].text, 445, 454, texts[10211][1].measurements_old_new.value[1][0]])

In [91]:
examples.append([texts[10795][1].text, 0, 26, 7.87])

In [92]:
examples.append([texts[11797][1].text, 0, 8, 3.89])

In [93]:
examples.append([texts[12184][1].text, 0, 26, 1.93])

In [94]:
examples.append([texts[14024][1].text, 9, 22, texts[14024][1].measurements_flat2.value[0][0]])

In [98]:
examples.append([texts[16101][1].text, 130, 168, texts[16101][1].measurements_old_new.value[0][0]])

In [99]:
examples.append([texts[16255][1].text, 13, 28, 0.005])

In [100]:
examples.append([texts[21664][1].text, 0, 17, 0.003])

In [101]:
examples.append([texts[22501][1].text, 'NA', 'NA', 'NA'])

In [103]:
examples.append([texts[25211][1].text, 27, 56, texts[25211][1].measurements_old_new.value[0][0]])

In [104]:
examples.append([texts[33042][1].text, 28, 33, 8])

In [105]:
examples.append([texts[39973][1].text, 119, 130, texts[39973][1].measurements_old_new.value[2][0]])

In [106]:
examples.append([texts[45612][1].text, 0, 25, 1.16])

In [107]:
examples.append([texts[61095][1].text, 0, 26, 0.136])

In [111]:
lines = ['PSA 03042012 - 0,83ng/ml perearsti poolt .',
 'PSA 2010. 3ng/ml, PSA 2012. 1,53ng/ml . - Bx va',
 'PSA 20105,99 ja 26.01.2012 uuesti .',
 'PSA 2011 oli 0 , 4 nG7ml .',
 'PSA 201222,25ng/ml',
 'PSA 2 aastajooksuldünaamikata , eriuuring',
 ':psa 16,81! ! ! ! ! ,',
 'Happe-aluse tasakaal 6.0 ( 5.0 .. 8.0 )',
 'loli 25 mgx1 ja Monoprili 10 mg Kolesterool 2011a',
 'Kolesterool 1k aastas .',
 'Kõrgenenud kolesterool 2a ( mõõdetud ). Ei pea dieetist kinni',
 'Kontr Verekol 08.12a Per-le juurde .',
 's vas munajuha kasvaja op , günekol 3a tagssi .',
 '08.11.2010 PSA 13.12.2011 7,2ng/ml PSADT on väike .',
 'Rütmihäire tsüklipikkus 330 msek',
 'Loote pikkus : \xa0 3 mm - vastab\xa0 5 nädalat 6 päeva.',
 'Põhjendus: PALAT 10 # ALAT maksanäitaja',
 'ärme vähk 2007 aastast cT3N0M0PSA 59ng/ml .',
 'PSA 8,5( püsib aastaid selles väärtus',
 'S,P-PSA 4.130( <4.100 µg/L )',
 'PSA 5,2.',
 'Kolesterool oli 7,9 mmol/l 0',
 'kolesterool 6.4.',
 'Kolesterool 5,2 mmol/l - esialgu dieet .',
 'SK 3900 g , SP 51 cm .',
 'Lapse kaal 5,4 kg/82 mg/0,82 ml i/m .',
 'Kehakaal 80,2 kg , KMI 25,9',
 'S,P-NT-proBNP 668 ( <125 pg/mL ) S,P-Albumiin 43 ( 35 .. 52 g/L ) S,P-ALAT 25 ( <33 U/L )',
 'PSA 6,5 ng/ml, eesnäärme maht67cm3',
 'rjeldus : Siinusbradükardia Fr 587min']

In [112]:
examples.append([lines[0], 0, 24, 0.83])
examples.append([lines[1], 0, 16, 3])
examples.append([lines[2], 0, 12, 5.99])
examples.append([lines[3], 0, 18, 0.4])
examples.append([lines[4], 0, 18, 22.25])
examples.append([lines[5], 'NA', 'NA', 'NA'])
examples.append([lines[6], 1, 10, 16.81])
examples.append([lines[7], 'NA', 'NA', 'NA'])
examples.append([lines[8], 'NA', 'NA', 'NA'])
examples.append([lines[9], 'NA', 'NA', 'NA'])
examples.append([lines[10], 'NA', 'NA', 'NA'])
examples.append([lines[11], 'NA', 'NA', 'NA'])
examples.append([lines[12], 'NA', 'NA', 'NA'])
examples.append([lines[13], 11, 34, 7.2])
examples.append([lines[16], 'NA', 'NA', 'NA'])
examples.append([lines[17], 30, 41, 59])
examples.append([lines[18], 0, 7, 8.5])
examples.append([lines[19], 0, 14, 4.13])
examples.append([lines[20], 0, 7, 5.2])
examples.append([lines[21], 0, 26, 7.9])
examples.append([lines[24], 0, 9, 3900])

In [114]:
import csv
with open("examples_for_testing.csv", "w") as fout:
    writer = csv.writer(fout)
    for line in examples:
        writer.writerow(line)

In [97]:
texts[16101][1].measurements_old_new[0]

text,start,end,name,key,value,unit,low,high,systolic,diastolic,pulse,year,month,day,hour,minute,prygi,subject
"fS,fP-Gluc 0 min (pre 75g Gluc PO) 8.0",130,168,GLÜKOOS,"fS,fP-Gluc",8.0,None,None,None,None,None,None,None,None,None,None,None,None,None


In [28]:
c = 0
for text_id, span_index, spans in sampler.sample_spans(50000, 'conflicts'):
    if spans[0].value == spans[1].value and spans[0].name == spans[1].name:
        continue
    if spans[0].name[0].strip() == 'VERERÕHK' and spans[1].name[0].strip() == 'VERERÕHK':
        continue
    if spans[0].name[0].strip() == 'PULSS' and spans[1].name[0].strip() == 'PULSS':
        continue    
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    c += 1
    spans[0].display()
    spans[1].display()

text_id: 49, span index: 0


text,start,end,span_status,input_layer_name,name,value
PSA 19 05,0,10,missing,measurements_old_new,PSA,1905


text,start,end,span_status,input_layer_name,name,value
"PSA 19 05 2011 -1,32",0,20,extra,measurements_flat2,PSA,1.32


text_id: 785, span index: 0


text,start,end,span_status,input_layer_name,name,value
PSA - 14.08,21,33,missing,measurements_old_new,PSA,14.08


...

text,start,end,span_status,input_layer_name,name,value
PSA 2 30,0,9,missing,measurements_old_new,PSA,230


text,start,end,span_status,input_layer_name,name,value
PSA 2,0,5,extra,measurements_flat2,PSA,2


text_id: 65801, span index: 0


text,start,end,span_status,input_layer_name,name,value
PSA 01 09,0,10,missing,measurements_old_new,PSA,0109


text,start,end,span_status,input_layer_name,name,value
PSA 01 09 2015 -1,0,17,extra,measurements_flat2,PSA,1


In [29]:
38214

134

In [14]:
for text_id, span_index, spans in sampler.sample_spans(2, 'overlapped'):
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    spans[0].display()
    spans[1].display()

text_id: 33766, span index: 0


text,start,end,span_status,input_layer_name,name,value
IPSS 19p,18,26,missing,measurements_old_flat,IPSS,19


text,start,end,span_status,input_layer_name,name,value
IPSS 19,17,25,extra,measurements_flat2,IPSS,19


text_id: 60712, span index: 0


text,start,end,span_status,input_layer_name,name,value
PSA 12 11,1,12,missing,measurements_old_flat,PSA,1211


text,start,end,span_status,input_layer_name,name,value
"PSA 12 11 2013 - 3,73",2,23,extra,measurements_flat2,PSA,3.73


In [ ]:
for text_id, span_index, spans in sampler.sample_spans(2, 'prolonged'):
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    spans[0].display()
    spans[1].display()

In [ ]:
for text_id, span_index, spans in sampler.sample_spans(2, 'shortened'):
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    spans[0].display()
    spans[1].display()

Display `measurements_diff_1_2` layer.

In [ ]:
from estnltk import Layer
Layer.print_start_end = True

texts = collection.select(layers=['measurement_tokens',
                                  'measurements_old',
                                  'measurements_flat',
                                  'measurements_diff_old_flat'],
                          keys=[4])
text_id, text = next(texts)
text.measurements_diff_old_flat

In [ ]:
storage.close()